##Installing and Mounting Media

In [1]:
# Install MediaPipe and OpenCV
!pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import os
from google.colab import drive

In [3]:
# Initialize MediaPipe pose and hands
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
pose = mp_pose.Pose(static_image_mode=True)  # Set static_image_mode for single images
hands = mp_hands.Hands(static_image_mode=True)
mp_drawing = mp.solutions.drawing_utils  # For drawing landmarks

In [4]:
# Mount Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# Define base directories for datasets
base_dir = '/content/gdrive/MyDrive/E-RAU(DB)/MA680/data/Shooting'
experienced_dir = os.path.join(base_dir, 'Experienced')
novice_dir = os.path.join(base_dir, 'Novice')

##Defining Functions and Classes

In [6]:
def process_and_save_frames(input_directory, output_directory):
    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # List all image files in the input directory
    image_files = [f for f in os.listdir(input_directory) if f.endswith(('.jpg', '.png'))]

    # Iterate through all image files
    for file_name in image_files:
        file_path = os.path.join(input_directory, file_name)
        print(f"Processing: {file_name}")

        # Read the image
        image = cv2.imread(file_path)
        if image is None:
            print(f"Warning: Could not load {file_name}")
            continue

        # Convert to RGB for MediaPipe processing
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Detect pose
        pose_results = pose.process(image_rgb)

        # Detect hands
        hand_results = hands.process(image_rgb)

        # Draw the results
        annotated_image = image.copy()
        if pose_results.pose_landmarks:
            mp_drawing.draw_landmarks(
                annotated_image,
                pose_results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS
            )
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    annotated_image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS
                )

        # Save the annotated image to the output directory
        output_path = os.path.join(output_directory, file_name)
        cv2.imwrite(output_path, annotated_image)
        print(f"Saved: {output_path}")

In [7]:
class ShootingFormDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []

        # Iterate over "No Error Detected" and "Error Detected" folders
        for label, folder in enumerate(["No Error Detected", "Error Detected"]):
            folder_path = os.path.join(data_dir, folder)
            for file in os.listdir(folder_path):
                if file.endswith(('.jpg', '.png')):
                    self.images.append(os.path.join(folder_path, file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        label = self.labels[idx]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image)

        return image, label

In [8]:
class AdvancedShootingFormClassifier(nn.Module):
    def __init__(self):
        super(AdvancedShootingFormClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.AdaptiveAvgPool2d((8, 8))
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

In [10]:
# Paths
data_dir = "/content/gdrive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/op_Processed_Frames"
model_save_path = "/content/gdrive/MyDrive/E-RAU(DB)/MA680/models/shooting_form_classifier_SFC.pth"

# Transforms
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Dataset and DataLoader
dataset = ShootingFormDataset(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AdvancedShootingFormClassifier().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

##Model Training

In [11]:
# Training Loop
epochs = 2
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update metrics
        running_loss += loss.item()
        predictions = (outputs > 0.5).float()
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

    # Calculate epoch metrics
    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = (correct_predictions / total_predictions) * 100
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

# Save the model
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Epoch 1/2, Loss: 0.3376, Accuracy: 85.71%
Epoch 2/2, Loss: 0.0800, Accuracy: 96.43%
Model saved to /content/gdrive/MyDrive/E-RAU(DB)/MA680/models/shooting_form_classifier_SFC.pth


##Model Applicaiton

In [16]:
# Load the trained model
model_path = "/content/gdrive/MyDrive/E-RAU(DB)/MA680/models/shooting_form_classifier_SFC.pth"
model = AdvancedShootingFormClassifier()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

frames_dir = "/content/gdrive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Extracted_Frames"
output_dir = "/content/gdrive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Processed_Frames"
os.makedirs(output_dir, exist_ok=True)

# Process each frame
for file_name in os.listdir(frames_dir):
    file_path = os.path.join(frames_dir, file_name)
    if not file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue  # Skip non-image files

    # Read and process the image
    image = cv2.imread(file_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect pose
    pose_results = pose.process(image_rgb)
    annotated_image = image.copy()

    if pose_results.pose_landmarks:
        # Draw the skeleton on the image
        mp_drawing.draw_landmarks(
            annotated_image,
            pose_results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS
        )

        # Convert skeleton data to input for the model
        landmarks = pose_results.pose_landmarks.landmark
        keypoints = torch.tensor([[lm.x, lm.y, lm.z, lm.visibility] for lm in landmarks])

        # Ensure the keypoints are reshaped into a 2D tensor: [1, 1, 33, 4]
        keypoints = keypoints.view(1, 1, 33, 4)

        # Expand to 3 channels if the model requires it
        keypoints = keypoints.repeat(1, 3, 1, 1)  # Convert to [batch_size, 3, 33, 4]

        # Prediction
        prediction = model(keypoints)
        label = "No Error Detected" if prediction.item() > 0.5 else "Error Detected"
    else:
        label = "Error Detected (No Pose Detected)"

    # Add label to the annotated image
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(annotated_image, label, (10, 30), font, 1, (0, 255, 0) if "No Error" in label else (0, 0, 255), 2)

    # Save the annotated image
    output_path = os.path.join(output_dir, file_name)
    cv2.imwrite(output_path, annotated_image)
    print(f"Processed and saved: {output_path}")

Processed and saved: /content/gdrive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Processed_Frames/Te11_2_frame4.jpg
Processed and saved: /content/gdrive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Processed_Frames/Te11_2_frame8.jpg
Processed and saved: /content/gdrive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Processed_Frames/Te11_2_frame5.jpg
Processed and saved: /content/gdrive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Processed_Frames/Te11_2_frame10.jpg
Processed and saved: /content/gdrive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Processed_Frames/Te11_2_frame6.jpg
Processed and saved: /content/gdrive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Processed_Frames/Te11_1_frame2.jpg
Processed and saved: /content/gdrive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Processed_Frames/Te10_2_frame10.jpg
Processed and saved: /content/gdrive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Processed